In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../../")
from experiment import Exp

from dataloaders import data_set,data_dict
import torch
import yaml
import os

from torch.optim import Adam

In [2]:
### Parameter for training

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict()   

args.to_save_path     = "/home/lukasprobst/Documents/Bachelorarbeit/Run_logs"           
args.freq_save_path   = "/home/lukasprobst/Documents/Bachelorarbeit/Freq_data"
args.window_save_path = "/home/lukasprobst/Documents/Bachelorarbeit/Sliding_window"
args.root_path        = "/home/lukasprobst/Documents/Bachelorarbeit/datasets"

args.drop_transition  = False
# Nomalization of the data, depends on dataset!
args.datanorm_type    = "standardization" # None ,"standardization", "minmax"

args.batch_size       = 64 # Reduce if not enough graphic memory
args.shuffle          = True # Give NN more variety to learn
args.drop_last        = True # Drop last batch if smaller than batch size
args.train_vali_quote = 0.90 # 90% training, 10% validation                     

# training setting 
args.train_epochs            = 60

args.learning_rate           = 0.001 # 10^(-3)
args.learning_rate_patience  = 20 # Reduce learning rate every 20 epochs
args.learning_rate_factor    = 0.1 # Reduce learning rate by 10%

args.early_stop_patience     = 15 # Stop after 15 epochs without improvement

args.use_gpu                 = True if torch.cuda.is_available() else False
args.gpu                     = 0 # Use first GPU
args.use_multi_gpu           = False # Use multiple GPUs

args.optimizer               = "Adam"
args.criterion               = "CrossEntropy"

# From ShaSpec paper:
# - 60 epochs
# - batch norm and dropout
# - adam optimizer with 10^(-2) weight decay
# - initial learning rate 10^(-3) and reduced by 10% every 20 epochs.

In [3]:
# Get content of folder args.window_save_path
files = os.listdir(args.window_save_path)

In [4]:
### Parameter for data

args.seed                             = 1 # Seed for reproducibility

args.data_name                        =  "dsads"

# Wavelet calculates frequence parameters from time series data, don't need that
args.wavelet_filtering                = False
args.wavelet_filtering_regularization = False
args.wavelet_filtering_finetuning     = False
args.wavelet_filtering_finetuning_percent = 0.5
args.wavelet_filtering_learnable      = False
args.wavelet_filtering_layernorm      = False

args.regulatization_tradeoff          = 0
args.number_wavelet_filtering         = 6

# Augmentation difference
args.difference          =  False

# Augmentation filtering
args.filtering           =  False

# Augmentation  magnitude
args.magnitude           =  False
args.weighted_sampler    = False

# MIXUP
args.mixup_probability  = 1
args.mixup_alpha  = 0.5

# Random Augmentation
args.random_augmentation_prob = 1
args.random_augmentation_config = {"jitter":True,
                                   "moving_average":True,
                                   "magnitude_scaling":True,
                                   "magnitude_warp":True,
                                   "magnitude_shift":True,
                                   "time_warp":True,
                                   "window_warp":True,
                                   "window_slice":True,
                                   "random_sampling":True,
                                   "slope_adding":True
                                   }
random_augmentation_nr = 0
for key in args.random_augmentation_config.keys():
    if args.random_augmentation_config[key]:
        random_augmentation_nr = random_augmentation_nr+1
args.random_augmentation_nr = random_augmentation_nr
args.max_aug = 3

args.pos_select       = None
args.sensor_select    = None

args.representation_type = "time"
args.exp_mode            = "LOCV"
if args.data_name      ==  "skodar":
    args.exp_mode            = "SOCV"
    
config_file = open('../../configs/data.yaml', mode='r')
data_config = yaml.load(config_file, Loader=yaml.FullLoader)
config = data_config[args.data_name]

args.root_path       = os.path.join(args.root_path,config["filename"])
args.sampling_freq   = config["sampling_freq"]
args.classes_num     =  config["num_classes"]
args.num_modalities  =  config["num_modalities"]
args.miss_rates       =  config["miss_rates"]
window_seconds       = config["window_seconds"]
args.windowsize      =   int(window_seconds * args.sampling_freq) 
args.input_length    =  args.windowsize
# Input information
# For ShaSpec we want the number of channels per modality
args.c_in            =  config["num_channels"] // args.num_modalities

if args.difference:
    args.c_in  = args.c_in * 2
    print(args.c_in )
if args.filtering:
    for col in config["sensors"]:
        if "acc" in col:
            args.c_in = args.c_in+1
            print(args.c_in )

if args.wavelet_filtering :
    if args.windowsize%2==1:
        N_ds = int(torch.log2(torch.tensor(args.windowsize-1)).floor()) - 2
    else:
        N_ds = int(torch.log2(torch.tensor(args.windowsize)).floor()) - 2

    args.f_in            =  args.number_wavelet_filtering*N_ds+1
else:
    args.f_in            =  1
            

In [5]:
### Parameter for the model

args.model_type = "shaspec"

args.activation = "ReLU"
args.decoder_type = "FC"
args.shared_encoder_type = "concatenated"
args.miss_rate = args.miss_rates[1] # 0.0 0.2 0.4 0.6 0.8

exp = Exp(args)

Use CPU
ShaSpec input shape:  (1, 1, 125, 9)
Build the ShaSpec model!
Done!
Parameter : 63944
Set the seed as :  1


In [6]:
exp.model

model_builder(
  (model): ShaSpec(
    (specific_encoders): ModuleList(
      (0-3): 4 x SpecificEncoder(
        (be): BaseEncoder(
          (conv_layers): Sequential(
            (0): Conv2d(1, 16, kernel_size=(5, 1), stride=(2, 1))
            (1): ReLU()
            (2): Conv2d(16, 16, kernel_size=(5, 1), stride=(2, 1))
            (3): ReLU()
            (4): Conv2d(16, 16, kernel_size=(5, 1), stride=(2, 1))
            (5): ReLU()
            (6): Conv2d(16, 16, kernel_size=(5, 1), stride=(2, 1))
            (7): ReLU()
          )
          (sa): SelfAttention(
            (query): Conv1d(16, 2, kernel_size=(1,), stride=(1,), bias=False)
            (key): Conv1d(16, 2, kernel_size=(1,), stride=(1,), bias=False)
            (value): Conv1d(16, 16, kernel_size=(1,), stride=(1,), bias=False)
          )
          (fc_fusion): Linear(in_features=80, out_features=32, bias=True)
        )
      )
    )
    (shared_encoder): SharedEncoder(
      (be): BaseEncoder(
        (conv_layer

In [7]:
exp.train()

================  Load all the data  ================
-----------------------Sliding file are generated -----------------------
-----------------------Sliding file are generated -----------------------
================  {dataset.exp_mode} Mode  ================
================  {dataset.num_of_cv} CV folds in total  ================
================ CV 0 ================
Leave one Out Experiment : The 1 Part as the test
[-] Target sampling weights:  [0.0026178  0.00273224 0.00260417 0.00269542 0.00268097 0.00265957
 0.0026178  0.0027027  0.00265252 0.0026455  0.00262467 0.00268097
 0.0025974  0.00263852 0.00262467 0.00265252 0.00268097 0.0025974
 0.00257069]
train
Train data number :  7182
vali
Validation data number :  798
test
Test data number :  1140
================ Build the shaspec model ================ 
ShaSpec input shape:  (1, 1, 125, 9)
Build the ShaSpec model!


Training Process [epochs]:   0%|          | 0/60 [00:00<?, ?it/s]

[(1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45)]
[2]
torch.Size([64, 1, 125, 9])
5
Missing modality:  tensor([[[[nan, nan, nan,  ..., nan, nan, nan],
  

[(1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45)]
[0]
torch.Size([64, 1, 125, 9])
5
Missing modality:  tensor([[[[nan, nan, nan,  ..., nan, nan, nan],
  

[(1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45)]
[2]
torch.Size([64, 1, 125, 9])
5
Missing modality:  tensor([[[[nan, nan, nan,  ..., nan, nan, nan],
  

...

[(1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45)]
[3]
torch.Size([64, 1, 125, 9])
5
Missing modality:  tensor([[[[nan, nan, nan,  ..., nan, nan, nan],
  

[(1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45)]
[1]
torch.Size([64, 1, 125, 9])
5
Missing modality:  tensor([[[[nan, nan, nan,  ..., nan, nan, nan],
  

Epoch 1/60, Loss: 0.4438:   0%|          | 0/113 [05:38<?, ?it/s]

[(1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45)]
[1]
torch.Size([14, 1, 125, 9])
5
Missing modality:  tensor([[[[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]]],


        [[[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]]],


        [[[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan

[(1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45), (1, 125, 45)]
[3]
torch.Size([64, 1, 125, 9])
5
Missing modality:  tensor([[[[nan, nan, nan,  ..., nan, nan, nan],
  